In [13]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [14]:
# Load the dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape the data into 4D tensors
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Convert the data to float32 and normalize it to values between 0 and 1
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [15]:
# Convert the labels to categorical one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [16]:
# Define the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [17]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Define the data augmentation generator
datagen = ImageDataGenerator(
        rotation_range=10,  # Rotate the image by a random degree between -10 and 10
        width_shift_range=0.1,  # Shift the image horizontally by a random fraction between -0.1 and 0.1 of the width
        height_shift_range=0.1,  # Shift the image vertically by a random fraction between -0.1 and 0.1 of the height
        zoom_range=0.1  # Zoom the image by a random factor between 0.9 and 1.1
)

# Fit the model with data augmentation
history_aug = model.fit(datagen.flow(x_train, y_train, batch_size=128),
                        steps_per_epoch=len(x_train) / 128, epochs=30,
                        validation_data=(x_test, y_test), callbacks=[EarlyStopping(patience=5)])

# Fit the model without data augmentation
history_no_aug = model.fit(x_train, y_train, batch_size=128, epochs=30,
                           validation_data=(x_test, y_test), callbacks=[EarlyStopping(patience=5)])

# Plot the training and validation accuracy for the augmented data
plt.plot(history_aug.history['accuracy'])
plt.plot(history_aug.history['val_accuracy'])
plt.title('Model accuracy with data augmentation')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot the training and validation accuracy for the non-augmented data
plt.plot(history_no_aug.history['accuracy'])
plt.plot(history_no_aug.history['val_accuracy'])
plt.title('Model accuracy without data augmentation')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
